In [9]:
from datasets import load_dataset

ds = load_dataset("VinitT/Sanskrit-Llama")
raw_text = "".join(ds['train']['input'])
raw_text[:100], len(raw_text)

('ये त्रि॑ष॒प्ताः प॑रि॒यन्ति॒ विश्वा॑ रू॒पाणि॒ बिभ्र॑तः । वा॒चस्पति॒र्बला॒ तेषां॑ त॒न्वो॑ अ॒द्य द॑धातु',
 2238578)

In [24]:
import re

# valid_chars_pattern = r'[a-zA-Z0-9अ-ह़-ी़॰ँ॒॑]+|[.,;!?()॥]|\s'
valid_chars_pattern = r'[\u0900-\u097F\u1CD0-\u1CFF\s]+|[॥।ʼ.,!?()"]'

filtered_text = re.sub(f'[^{valid_chars_pattern}]', '', raw_text)

In [25]:
len(raw_text), len(filtered_text)

(2238578, 2220079)

In [26]:
diff_text = [x for x in set(raw_text) if x not in set(filtered_text)]
diff_text[:20]

['\uf184',
 '\uf130',
 '\uf1a3',
 '\uf11e',
 '-',
 '(',
 'ꣳ',
 '\u200c',
 '\uf185',
 '̫',
 '\uf1a2',
 ':',
 ']',
 '3',
 ',',
 ')',
 '\uf15c',
 '\u200d',
 '.']

In [42]:
filtered_text[:1000]

'ये त्रि॑ष॒प्ताः प॑रि॒यन्ति॒ विश्वा॑ रू॒पाणि॒ बिभ्र॑तः । वा॒चस्पति॒र्बला॒ तेषां॑ त॒न्वो॑ अ॒द्य द॑धातु मे  ॥ १पुन॒रेहि॑ वाचस्पते दे॒वेन॒ मन॑सा स॒ह । वसो॑ष्पते॒ नि र॑मय॒ मय्ये॒वास्तु॒ मयि॑ श्रु॒तम्  ॥ २इ॒हैवाभि वि त॑नू॒भे आर्त्नी॑ इव॒ ज्यया॑ । वा॒चस्पति॒र्नि य॑च्छतु॒ मय्ये॒वास्तु॒ मयि॑ श्रु॒तम्  ॥ ३उप॑हूतो वा॒चस्पति॒रुपा॒स्मान्वा॒चस्पति॑र्ह्वयताम् । सं श्रु॒तेन॑ गमेमहि॒ मा श्रु॒तेन॒ वि रा॑धिषि  ॥ ४वि॒द्मा श॒रस्य॑ पि॒तरं॑ प॒र्जन्यं॒ भूरि॑धायसम् । वि॒द्मो ष्व॑स्य मा॒तरं॑ पृथि॒वीं भूरि॑वर्पसम्  ॥ १ज्या॑के॒ परि॑ णो न॒माश्मा॑नं त॒न्वं कृधि । वी॒डुर्वरी॒यो ऽरा॑ती॒रप॒ द्वेषां॒स्या कृ॑धि  ॥ २वृ॒क्षं यद्गावः॑ परिषस्वजा॒ना अ॑नुस्फु॒रं श॒रमर्च॑न्त्यृ॒भुम् । शरु॑म॒स्मद्या॑वय दि॒द्युमि॑न्द्र  ॥ ३यथा॒ द्यां च॑ पृथि॒वीं चा॒न्तस्तिष्ठ॑ति॒ तेज॑नम् । ए॒वा रोगं॑ चास्रा॒वं चा॒न्तस्ति॑ष्ठतु॒ मुञ्ज॒ इत्  ॥ ४वि॒द्मा श॒रस्य॑ पि॒तरं॑ प॒र्जन्यं॑ श॒तवृ॑ष्ण्यम् । तेना॑ ते त॒न्वे॑३ शं क॑रं पृथि॒व्यां ते॑ नि॒षेच॑नं ब॒हिष्टे॑ अस्तु॒ बालिति॑  ॥ १वि॒द्मा श॒रस्य॑ पि॒तरं॑ मि॒त्रं श॒तवृ॑ष्ण्यम् । तेना॑ ते त॒न्वे॑३ शं क॑रं 

In [43]:
text = filtered_text
# Step 1: Create a vocabulary for characters
vocab = {char: idx for idx, char in enumerate(set(text), start=0)}
reverse_vocab = {idx: char for char, idx in vocab.items()}

# Convert text to initial tokens (character-level tokenization)
tokens = [vocab[char] for char in text]

In [45]:
len(vocab)

80

In [46]:
# Helper functions for BPE
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx, reverse_map):
    new_ids = []
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
            new_ids.append(idx)
            reverse_map[idx] = reverse_map[pair[0]] + reverse_map[pair[1]]
            i += 2
        else:
            new_ids.append(ids[i])
            i += 1
    return new_ids

def decode_ids(ids, reverse_map):
    """Convert token IDs back to their character or merged string representation."""
    return [reverse_map[token] for token in ids]


In [64]:
# Step 2: Perform BPE
vocab_size = 5000
num_merges = vocab_size - len(vocab)

ids = tokens[:]
merges = {}
reverse_merges = {idx: reverse_vocab[idx] for idx in ids}  # Initialize reverse map

from tqdm import tqdm

for i in tqdm(range(num_merges)):
    stats = get_stats(ids)
    if not stats:
        break
    pair = max(stats, key=stats.get)
    idx = max(reverse_merges.keys()) + 1
    ids = merge(ids, pair, idx, reverse_merges)
    merges[pair] = idx

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4920/4920 [17:34<00:00,  4.67it/s]


In [51]:
tokens[:5], ids[:5]

([65, 50, 63, 78, 62], [65, 99, 150, 98, 9])

In [101]:
print(f"Compression ration: {round(len(tokens)/len(ids),1)}")

Compression ration: 3.3


In [65]:
len(tokens)/len(ids)

3.2666956048321834

In [70]:
decode_ids(tokens[:50], reverse_merges)

['य',
 'े',
 ' ',
 'त',
 '्',
 'र',
 'ि',
 '॑',
 'ष',
 '॒',
 'प',
 '्',
 'त',
 'ा',
 'ः',
 ' ',
 'प',
 '॑',
 'र',
 'ि',
 '॒',
 'य',
 'न',
 '्',
 'त',
 'ि',
 '॒',
 ' ',
 'व',
 'ि',
 'श',
 '्',
 'व',
 'ा',
 '॑',
 ' ',
 'र',
 'ू',
 '॒',
 'प',
 'ा',
 'ण',
 'ि',
 '॒',
 ' ',
 'ब',
 'ि',
 'भ',
 '्',
 'र']

In [71]:
decode_ids(ids[:50], reverse_merges)

['ये ',
 'त्र',
 'ि॑ष',
 '॒प्',
 'ता',
 'ः प॑र',
 'ि॒य',
 'न्ति॒ ',
 'विश्वा॑ ',
 'रू॒पा',
 'णि॒ ',
 'बिभ्र',
 '॑तः । ',
 'वा॒च',
 'स्प',
 'ति॒र्',
 'बल',
 'ा॒ ',
 'ते',
 'षां॑ ',
 'त॒न्व',
 'ो॑ अ॒',
 'द्य ',
 'द॑धा',
 'तु ',
 'मे',
 '  ॥ १',
 'पुन',
 '॒रे',
 'हि॑ ',
 'वाच',
 'स्पते ',
 'दे॒व',
 'ेन॒ ',
 'मन॑स',
 'ा स',
 '॒ह । ',
 'वस',
 'ो॑',
 'ष्प',
 'ते॒ ',
 'नि ',
 'र॑म',
 'य॒ ',
 'मय',
 '्ये',
 '॒वा',
 'स्तु॒ ',
 'मयि॑ ',
 'श्रु॒त']

In [72]:
vocab

{'ऽ': 0,
 'ल': 1,
 'ं': 2,
 'ए': 3,
 'ौ': 4,
 '९': 5,
 'ू': 6,
 'ा': 7,
 '᳕': 8,
 'ष': 9,
 'ो': 10,
 '᳡': 11,
 'ृ': 12,
 'ओ': 13,
 'ट': 14,
 'ै': 15,
 'ख': 16,
 'र': 17,
 'न': 18,
 '०': 19,
 'ळ': 20,
 'ठ': 21,
 'ॐ': 22,
 'ड': 23,
 '६': 24,
 'छ': 25,
 '४': 26,
 'आ': 27,
 'प': 28,
 '॥': 29,
 'औ': 30,
 '८': 31,
 'ज': 32,
 'ऊ': 33,
 'ॢ': 34,
 'घ': 35,
 'ः': 36,
 'व': 37,
 'ढ': 38,
 '३': 39,
 'ि': 40,
 'द': 41,
 'ु': 42,
 'म': 43,
 'फ': 44,
 'थ': 45,
 'ी': 46,
 'अ': 47,
 'ई': 48,
 '२': 49,
 'े': 50,
 'स': 51,
 'च': 52,
 '॒': 53,
 'ँ': 54,
 'ब': 55,
 'इ': 56,
 'श': 57,
 'ह': 58,
 'ऐ': 59,
 'उ': 60,
 '।': 61,
 '्': 62,
 ' ': 63,
 'ङ': 64,
 'य': 65,
 '॑': 66,
 'ण': 67,
 '१': 68,
 'ञ': 69,
 'ऋ': 70,
 'ॄ': 71,
 'क': 72,
 'झ': 73,
 '७': 74,
 '५': 75,
 'ग': 76,
 'ध': 77,
 'त': 78,
 'भ': 79}

In [73]:
len(ids) # token >> ids

679610

In [77]:
len(set(tokens)), len(set(ids)), len(ids)

(80, 4980, 679610)

In [84]:
decode_ids(list(set(tokens))[-5:], reverse_map=reverse_merges)

['५', 'ग', 'ध', 'त', 'भ']

In [83]:
decode_ids(list(set(ids))[-5:], reverse_map=reverse_merges)

['वस्तो', 'श्र॑य', 'पुरो॒', 'दश॑ ', '॑म॒भि ']

In [88]:
reverse_merges[list(set(ids))[-1]]

'॑म॒भि '

In [95]:
reverse_merges

{65: 'य',
 50: 'े',
 63: ' ',
 78: 'त',
 62: '्',
 17: 'र',
 40: 'ि',
 66: '॑',
 9: 'ष',
 53: '॒',
 28: 'प',
 7: 'ा',
 36: 'ः',
 18: 'न',
 37: 'व',
 57: 'श',
 6: 'ू',
 67: 'ण',
 55: 'ब',
 79: 'भ',
 61: '।',
 52: 'च',
 51: 'स',
 1: 'ल',
 2: 'ं',
 10: 'ो',
 47: 'अ',
 41: 'द',
 77: 'ध',
 42: 'ु',
 43: 'म',
 29: '॥',
 68: '१',
 58: 'ह',
 49: '२',
 56: 'इ',
 15: 'ै',
 27: 'आ',
 46: 'ी',
 32: 'ज',
 25: 'छ',
 39: '३',
 60: 'उ',
 76: 'ग',
 26: '४',
 12: 'ृ',
 45: 'थ',
 72: 'क',
 23: 'ड',
 0: 'ऽ',
 44: 'फ',
 21: 'ठ',
 3: 'ए',
 69: 'ञ',
 14: 'ट',
 75: '५',
 24: '६',
 74: '७',
 31: '८',
 5: '९',
 33: 'ऊ',
 48: 'ई',
 16: 'ख',
 4: 'ौ',
 54: 'ँ',
 59: 'ऐ',
 70: 'ऋ',
 64: 'ङ',
 35: 'घ',
 38: 'ढ',
 13: 'ओ',
 71: 'ॄ',
 19: '०',
 11: '᳡',
 30: 'औ',
 22: 'ॐ',
 20: 'ळ',
 73: 'झ',
 34: 'ॢ',
 8: '᳕',
 80: '॒ ',
 81: '॑ ',
 82: '्य',
 83: '्र',
 84: '्व',
 85: 'न्',
 86: 'ा ',
 87: '। ',
 88: 'ं ',
 89: '॒त',
 90: '  ',
 91: 'ि॒',
 92: 'र्',
 93: ' । ',
 94: 'ा॑',
 95: 'स्',
 96: 'ा॒',
 97: 'ो ',
 98: 'ि॑',


In [92]:
[x for x in range(5000) if x not in list(set(ids))]

[29,
 136,
 201,
 1251,
 1832,
 1833,
 1853,
 2941,
 3390,
 3912,
 3913,
 4135,
 4567,
 4654,
 4720,
 4721,
 4869,
 4872,
 4874,
 4875]

In [ ]:
new_vocab = { for x in sorted(list(set(ids)))}